In [1]:
import pandas as pd
import spacy
import spacy_transformers
import pathlib
import os
import fr_dep_news_trf
#import openai #!pip install openai
# Il faut aller chercher le token de openai pour utiliser leur API
# L'idée est de constituer un résumé pour chacun des documents et utiliser un moteur de recherche par mot clé sur les textes ?
# Ou sinon uniquement par tag ?


nlp_max = fr_dep_news_trf.load()
nlp_eff = spacy.load("fr_core_news_sm")

In [3]:
df = pd.read_csv("./data/data_article_clean.csv")
df

,Unnamed: 0.1,Unnamed: 0,URL,TITRE,IDinDB,TYPE,RESUME,ARTICLE,DATE,YEAR
0,2708,2708,https://www.millenaire3.com/ressources/L-accom...,L'accompagnement des personnes en deuil : qual...,10055,Étude,Au fil des évolutions des sociétés occidentale...,Sommaire : 1 - Besoin d’information et de tran...,2013-09-30,2013
1,1394,1394,https://www.millenaire3.com/ressources/deux-mi...,Deux mille ans de culture à Lyon,10096,Texte,NaN,Lyon n’a connu dans son histoire que quelques ...,2000-01-01,2000
2,1395,1395,https://www.millenaire3.com/ressources/modelis...,Modéliser pour gérer la ville durable,10143,Texte,Article écrit pour la revue M3 n°5\r\nLa ville...,"Le projet Vélino’v, démarré en février 2013, i...",2013-01-06,2013
3,1401,1401,https://www.millenaire3.com/ressources/la-vill...,La ville comme système complexe,10184,Texte,Article écrit pour la revue M3 n°5\r\nLes scie...,"L’étude des systèmes complexes, interdisciplin...",2013-01-06,2013
4,2061,2061,https://www.millenaire3.com/ressources/le-role...,Le rôle de l’institution hospitalière dans l’u...,10240,Texte,NaN,"Anne-Sophie Clemençon, historienne de l’archit...",2002-01-01,2002
...,...,...,...,...,...,...,...,...,...,...
2769,2770,2770,https://www.millenaire3.com/ressources/2022/sp...,"Spectacles vivants et technologies numériques,...",NaN,Article,"Depuis le siècle dernier, le numérique s’est f...",L’histoire des arts numériques trace un dialog...,2022-01-31,2022
2770,2771,2771,https://www.millenaire3.com/ressources/2022/st...,Stratégie numérique des établissements d’arts ...,NaN,Article,"Site internet, communication sur les réseaux s...",Comment donner sa place au virtuel dans quelqu...,2022-07-02,2022
2771,2776,2776,https://www.millenaire3.com/ressources/2023/je...,"Jeunesse, culture & numérique : les six grands...",NaN,Article,De l’adolescence à l’entrée dans la vie active...,En collaboration avec la direction de la Cultu...,2023-05-05,2023
2772,2777,2777,https://www.millenaire3.com/ressources/danser-...,Danser sur les places : les passions urbaines ...,NaN,Texte,Histoire du danseur Pierre Deloche et de son i...,"Pendant plus de 15 ans, Pierre Deloche a dansé...",2000-01-01,2000


In [10]:
def get_lemma(nlp):

    rows, cols = df.shape 

    for row in range(rows):
        txt = []
        art = str(df.iloc[row].ARTICLE)
        res = str(df.iloc[row].RESUME)
        titre = str(df.iloc[row].TITRE)
        id = df.iloc[row]["Unnamed: 0"]
        
        full = titre+'\n'+res+'\n'+art
        doc2 = nlp(res+'\n'+art)
        
        for token in doc2:
            if token.is_stop == False:
                txt.append(token.lemma_)
            
        pathlib.Path(f'./data/Texte/{id}').mkdir(parents=True, exist_ok=True)
        with open(f"./data/Texte/{id}/full.txt", "w", encoding="utf8") as f:
            f.write(full)
        with open(f"./data/Texte/{id}/texte.txt", "w", encoding="utf8") as f:
            f.write(" ".join(txt))
        
        print(f"\rSuccessfuly exported {row} / {rows} : {txt[0:10]}", end="")
    
    

In [11]:
get_lemma(nlp_eff)

Successfuly exported 57 / 2774 : ['an', 'mise', 'service', '(', 'mai', '2005', ')', ',', 'vélo', 'rouge']uestion', 'voile']'.', 'vise']', 'solide', 'dépassionner']